In [ ]:
# Import modules

import sys
import pandas as pd
import numpy as np
import random
import pickle
import datetime
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# Open .sav data, correct variable names
# 13:26
# import savReaderWriter as spss

# sav_data = spss.SavReader('X:/Brondata/donatiedata_200601_201902.sav', returnHeader = True)
# raw_data = pd.DataFrame(list(sav_data))
raw_data.columns = ['DonatieEIN', 'DonatieDatum', 'RegistratieTijd', 'Ferritine', 'FerritineDatum', 'Donatiesoortcode', 'Donatiestatuscode', 'Volume',
                    'DonatieStarttijd', 'DonatieStoptijd', 'WO_Antwoord', 'Hb', 'KeyID', 'Geslacht', 'Gewicht', 'Lengte', 'Bloedgroep', 'Geboortejaar']

raw_data.drop([0], inplace=True)
raw_data.reset_index(inplace=True, drop=True)

In [ ]:
raw_data.head()

In [ ]:
# Make donations dataframe
cols = ['KeyID', 'DonatieDatum', 'DonatieStarttijd', 'DonatieStopttijd', 'Donatiesoortcode', 'Volume', 'Hb', 'Ferritine', 'FerritineDatum']
df = raw_data.loc[:, cols]

# Correct encoding
for cname in ['KeyID', 'DonatieDatum', 'DonatieStarttijd', 'DonatieStopttijd', 'Donatiesoortcode', 'FerritineDatum']:
    df.loc[:, cname] = df.loc[:, cname].str.decode("utf-8")

# Transform variables
df['DonatieDatum'] = pd.to_datetime(df['DonatieDatum'], errors='coerce', format='%Y/%m/%d')
df['DonatieStarttijd'] = pd.to_datetime(df['DonatieStarttijd'], errors='coerce', format='%H:%M:%S').dt.time
df['DonatieStopttijd'] = pd.to_datetime(df['DonatieStopttijd'], errors='coerce', format='%H:%M:%S').dt.time
df['FerritineDatum'] = pd.to_datetime(df['FerritineDatum'], errors='coerce', format='%Y/%m/%d')
df['KeyID'] = df['KeyID'].astype('category')
df['Donatiesoortcode'] = df['Donatiesoortcode'].astype('category')
df['DateTime'] = ''
df.loc[:, 'DateTime'] = pd.to_datetime(df.loc[:, 'DonatieDatum'].apply(str) + ' ' + df.loc[:, 'DonatieStarttijd'].apply(str))

for index, row in df.loc[df['Volume'] < 0, :].iterrows():
    df.loc[index, 'Volume'] = 0
df['Volume'] = pd.to_numeric(df['Volume'], downcast='unsigned', errors='coerce')
df.loc[np.isnan(df['Volume']), 'Volume'] = 0
df['Hb'] = pd.to_numeric(df['Hb'], errors='coerce')
df['Ferritine'] = pd.to_numeric(df['Ferritine'], errors='coerce')

# Save dataframe
df.to_pickle('X:/donaties.pkl')

# Make donor dataframe
cols = ['KeyID', 'Geboortejaar', 'Geslacht', 'Bloedgroep', 'Gewicht', 'Lengte']
df_donors = raw_data.loc[:, cols]

# Correct encoding
for cname in ['KeyID', 'Geslacht', 'Bloedgroep']:
    df_donors.loc[:, cname] = df_donors.loc[:, cname].str.decode("utf-8")
    
# Drop duplicate entries per donor, check number of donors
df_donors.drop_duplicates(keep='first', inplace=True)
print(len(np.unique(df_donors['KeyID'])), "unique donors")
print(df_donors.shape[0], "total in dataframe")

# Set index to donor ID
df_donors.set_index('KeyID')

# Save dataframe
df_donors.to_pickle('X:/donors.pkl')

In [ ]:
df_donors.head()

In [ ]:
# List of KeyIDs by sex
id_f = list(df_donors.loc[df_donors['Geslacht'] == 'V', ].index)
id_m = list(df_donors.loc[df_donors['Geslacht'] == 'M', ].index)

# Add column HbCheck 
conditions = [(df['KeyID'].isin(id_f)) & (df['Hb'] >= 7.8) & (df['Hb'] <= 11),
              (df['KeyID'].isin(id_f)) & (df['Hb'] < 7.8),
              (df['KeyID'].isin(id_f)) & (df['Hb'] > 11),
              (df['KeyID'].isin(id_m)) & (df['Hb'] >= 8.4) & (df['Hb'] <= 12),
              (df['KeyID'].isin(id_m)) & (df['Hb'] < 8.4),
              (df['KeyID'].isin(id_m)) & (df['Hb'] > 12)]
choices = ['good', 'low', 'high', 'good', 'low', 'high']

df['HbLowHigh'] = ''
df.loc[:, 'HbLowHigh'] = np.select(conditions, choices)

# Save dataframe
df.to_pickle('X:/donaties.pkl')

In [ ]:
# Preprocessing for time series

df = pd.read_pickle('X:/Brondata/donatiedata_200601_201902.pkl')
df.head()

# Read donor data
df_donors = pd.read_pickle('X:/Brondata/donors.pkl')

# List of KeyIDs by sex
id_f = list(df_donors.loc[df_donors['Geslacht'] == 'V', ].index)
id_m = list(df_donors.loc[df_donors['Geslacht'] == 'M', ].index)

start = datetime.datetime.now()

# Get time relative to first measurement
counter = 0
df_rel = pd.DataFrame(columns=list(df.columns))
for s_id in np.unique(list(df['KeyID'])):
    df_sub = df[df['KeyID'] == s_id].sort_values('DateTime').reset_index(drop=True).drop_duplicates('DateTime')
    tzero = df_sub.loc[0, 'DateTime']
    df_sub['TimeSinceFirst'] = df_sub['DateTime'] - tzero
    df_rel = df_rel.append(df_sub, sort=True)
    counter += 1
    if counter in [10, 100, 1000, 10000, 100000]:
        print(datetime.datetime.now())
        print(counter)
    if (datetime.datetime.now() - start) / pd.Timedelta('1 min') > 15:
        print(datetime.datetime.now())
        print(counter)
        start = datetime.datetime.now()
    if counter % 100000 == 0:
        df_rel.to_pickle('X:/df_rel_full.pkl')
        
df_rel.loc[df_rel['Hb'] == 999, 'Hb'] = None

In [ ]:
dfmini = df.loc[df['KeyID'].isin(np.unique(df['KeyID'])[:3])]
dfmini.head()

In [ ]:
type(df.loc[0, 'DateTime'])

In [ ]:
data2['diffs'] = data2.groupby('ticker')['value'].diff()

In [ ]:
demean = lambda df: df - df.mean()
df.groupby(['Groupvar1', 'Groupvar2']).transform(demean)

In [ ]:
def scaleDate(df):
    print(df)
#     tzero = df.loc[0, 'DateTime']
#     df['DateTime'] = df['DateTime'] - tzero
#     return df

In [ ]:
t = dfmini.loc[dfmini['KeyID'] == 'DK.00000005', ]

In [ ]:
t['DateTime']

In [ ]:
df = pd.read_pickle('X:/df_rel_full.pkl')

In [ ]:
df.head()

In [ ]:
max(df['DateTime'])